In [1]:
using PyPlot
using LinearAlgebra
using JuMP
using Ipopt
using Statistics
using RandomMatrices
using BirkhoffDecomposition
using Permutations

In [18]:
function rowsum(A) #Returns a column vector containing the row sums of square matrix A
    m = size(A,1)
    rowsum = zeros(m,1)
    for i in 1:m
        rowsum[i] = sum(A[i,:])
    end
    return rowsum
end
####################################
function colsum(A) #Returns a column vector containing the column sums of square matrix A
    n = size(A,2)
    colsum = zeros(n,1)
    for i in 1:n
        colsum[i] = sum(A[:,i])
    end
    return colsum
end
####################################
function rs_ize(A) #Returns a row stochastic matrix, given a nonnegative matrix A
    rsA = rowsum(A)
    A = inv(diagm(vec(rsA)))*A  #A is row stochastic
    return A
end
####################################
# Φ cal with sigmoid
function Φ(dif,F,W)
    sig1 = 1-(1+exp(F*(dif+W)))^(-1)
    sig2 = (1+exp(F*(dif-W))).^(-1)
    return sig1 .* sig2
end
######################################
# HK model (for update)
function HK_update(A,x_os_up,F,W)
    N_a = size(A,2)
    xup = zeros(N_a,1)
    confi = 0;
    update = 0;
    for i in 1:N_a
        confi = sum(A[i,j]*Φ(x_os_up[j,1]-x_os_up[i,1],F,W) for j in 1:N_a)
        update = sum(A[i,j]*(Φ(x_os_up[j,1]-x_os_up[i,1],F,W)*(x_os_up[j,1]-x_os_up[i,1])) for j in 1:N_a)
        xup[i,1] = x_os_up[i,1] + (1/confi)*(update)
        update = 0
        confi = 0
    end 
    return xup
end
###################################
#Projection of vector x_os onto the hypercube [-1,1]^n
function hyp_proj(x_os)
    for i in 1:length(x_os)
        if x_os[i] >= 1
            x_os[i] = 1
        end
        if x_os[i] <= -1
            x_os[i] = -1
        end
    end
    return x_os
end
####################################
#Control of opinion dynamics control in social network
####################################
#COD = controlled opinion dynamics
#OSA = one step ahead
#x_os = current state 
#targ = vector of targets for 1 player
#b = input coupling vector for 1 player
#This function is set up to update agent states under the action of one player
#Written for the case of one player at a time, OSA, Gauss-Seidel updates
function COD_OSA_J_DeGroot(x_init,player_targets,γ,K_f,A,B,N_a,N_p,proj_flag)
#Initialize arrays
    #N_a = length(x_init) #Number of agents
    #N_p = length(player_targets) #Number of players
    x_os = zeros(N_a,K_f + 1)  #Column j: opinions of N_a agents at instant j, Row i: time evolution of agent i's opinion
    u_os = zeros(N_p,K_f)      #Column j: controls of N_p players at instant j, Row i: time evolution of player i's controls
    x_os[:,1] = x_init         #Initial states (opinions) of each agent
    temp = zeros(N_a,1)        #Temporary vector to projection  
    for k in 1:K_f #Time loop
            for p in 1:N_p #player loop
            
                #Name the model, define the solver and set its attributes
                S = "$p"
                S = Model(Ipopt.Optimizer)
                set_optimizer_attributes(S, "print_level" => 0 )
            
                #Define the optimization variables
                @variable(S, v) #v is the scalar value of the control for player p
                @variable(S, xn[1:N_a,1:1]) #xn is the new state of agent influenced by p
            
                #Define the control constraints
                #@constraint(S, u_min <= v <= u_max)
            
                #Define the dynamical constraint
                @constraint(S, xn .== A*x_os[:,k] + B[:,p]*v)
                #@constraint(S, 0 .<= xn .<= 1 )
                #Define the objective function for each agent
                @objective(S, Min, sum((xn - player_targets[:,p]).^2) + γ[p]*v*v)
            
                optimize!(S)
            
                #Retrieve optimal values for k-th iteration (instant)
                u_os[p,k] = JuMP.value.(S[:v])
            end
        temp[:,1] = A*x_os[:,k] + B*u_os[:,k]
        
        if proj_flag == 1
            x_os[:,k+1] = hyp_proj(temp[:,1])
        else
            x_os[:,k+1] = temp[:,1]
        end
    end
    return x_os, u_os
end
############################################################
#COD = controlled opinion dynamics, J = Jacobi information exchange; targets for players are specified
function COD_OSA_J_targets_FJ(x_init,player_targets,γ,K_f,A,B,Θ,N_a,N_p,proj_flag)
#Initialize arrays
    #N_a = length(x_init) #Number of agents
    #N_p = length(player_targets) #Number of players
    x_os = zeros(N_a,K_f + 1)  #Column j: opinions of N_a agents at instant j, Row i: time evolution of agent i's opinion
    u_os = zeros(N_p,K_f)      #Column j: controls of N_p players at instant j, Row i: time evolution of player i's controls
    x_os[:,1] = x_init         #Initial states (opinions) of each agent
    temp = zeros(N_a,1)        #Temporary vector to projection  
    for k in 1:K_f #Time loop
            for p in 1:N_p #player loop
            
                #Name the model, define the solver and set its attributes
                S = "$p"
                S = Model(Ipopt.Optimizer)
                set_optimizer_attributes(S, "print_level" => 0 )
            
                #Define the optimization variables
                @variable(S, v) #v is the scalar value of the control for player p
                @variable(S, xn[1:N_a,1:1]) #xn is the new state of agent influenced by p
            
                #Define the control constraints
                #@constraint(S, u_min <= v <= u_max)
            
                #Define the dynamical constraint
                @constraint(S, xn .== A*x_os[:,k] + Θ*x_init + B[:,p]*v)
                #@constraint(S, 0 .<= xn .<= 1 )
                #Define the objective function for each agent
                @objective(S, Min, sum((xn - player_targets[:,p]).^2) + γ[p]*v*v)
            
                optimize!(S)
            
                #Retrieve optimal values for k-th iteration (instant)
                u_os[p,k] = JuMP.value.(S[:v])
            end
        temp[:,1] = A*x_os[:,k] + Θ*x_init + B*u_os[:,k]
        
        if proj_flag == 1
            x_os[:,k+1] = hyp_proj(temp[:,1])
        else
            x_os[:,k+1] = temp[:,1]
        end
    end
    return x_os, u_os
end

############################################################
# HK with control using Jacobi update
function COD_OSA_J_targets_HK(x_init,targets,γ,K_f,A,B,F,W,flag_proj)
    #Initialize arrays
    N_a = size(A,1)
    N_p = size(B,2)
    x_os = zeros(N_a,K_f + 1)  #Column j: opinions of N_a agents at instant j, Row i: time evolution of agent i's opinion
    x_os[:,1] = x_init         #Initial states (opinions) of each 
    u_os = zeros(N_p,K_f)      #Column j: controls of N_p players at instant j, Row i: time evolution of player i's controls    
    
    for k in 1:K_f #Time loop
            for p in 1:N_p #player loop
                #Name the model, define the solver and set its attributes
                S = "$p"
                S = Model(Ipopt.Optimizer)
                set_optimizer_attributes(S, "print_level" => 0 )
            
                #Define the optimization variables
                @variable(S, v) #v is the scalar value of the control for player p
                @variable(S, xn[1:N_a,1:1]) #xn is the new state of agent influenced by p
            
                #Define the control constraints
                #@constraint(S, u_min <= v <= u_max)
            
                #Define the dynamical constraint
                @constraint(S, xn .== HK_update(A,x_os[:,k],F,W) + B[:,p]*v)
                #@constraint(S, 0 .<= xn .<= 1 ) #Restrição nas opiniões
                #Define the objective function for each agent
                @objective(S, Min, sum((xn - targets[:,p]).^2) + γ[p]*v*v)
            
                optimize!(S)
            
                #Retrieve optimal values for k-th iteration (instant)
                u_os[p,k] = JuMP.value.(S[:v])
            end
        
        x_int = HK_update(A,x_os[:,k],F,W) + B*u_os[:,k]
        
        if flag_proj == 1
            x_os[:,k+1] = hyp_proj(x_int)
        else
            x_os[:,k+1] = x_int
        end
    end
    
    return x_os, u_os
end
############################################################

function plot_opinions(x_os,N)
    figure(figsize=(6,6))
    grid()
    for i in 1:size(x_os,1)
        plot(x_os[i,1:N],".-")
    end
    #=
    plot(x1_hat_4.*ones(K_f_4,1),"--",color = "blue");
    plot(x2_hat_4.*ones(K_f_4,1),"--",color = "darkorange");
    plot(x3_hat_4.*ones(K_f_4,1),"--",color = "green");
    =#
    xlabel("Time")
    ylabel("Opinions \$x_i\$, \$i=1,...,5\$")
    #legend([L"x_1",L"x_2", L"x_3", L"x_4", L"x_5"],loc="best")
    legend([L"x_1",L"x_2", L"x_3", L"x_4", L"x_5",L"x_6",L"x_7", L"x_8", L"x_9", L"x_{10}"],loc="best")
    title("Time plots of opinion: DeGroot+Control model, Gauss-Seidel proc.")
end

###########################################################
function plot_targ_5a3p(x_plot, target, T, model, save)
    figure(figsize=(6,6))
    grid()
    if model == 1
        for i in 1:5
            plot(x_plot[i,1:T],".-")
        end
    elseif model == 2
        for i in 6:10
            plot(x_plot[i,1:T],".-")
        end
    elseif model == 3
        for i in 11:15
            plot(x_plot[i,1:T],".-")
        end
    end
    plot(target[1].*ones(T,1),"--",color = "blue");
    plot(target[2].*ones(T,1),"--",color = "darkorange");
    plot(target[3].*ones(T,1),"--",color = "green");
    xlabel("Time")
    ylabel("Opinions \$x_i\$")
    legend([L"x_1",L"x_2", L"x_3", L"x_4", L"x_5",L"goal_1",L"goal_2", L"goal_3"],loc="right")
    
    if model == 1
        title("Time plots of opinion: dGc, Jacobi procedure")
        if save == 1
            savefig("DeGroot_Jacobi_5players.png")
            savefig("DeGroot_Jacobi_5players.eps")
        end
    elseif model == 2
        title("Time plots of opinion: FJc, Jacobi procedure")
        if save == 1
            savefig("FJ_Jacobi_5players.png")
            savefig("FJ_Jacobi_5players.eps")
        end
    elseif model == 3
        title("Time plots of opinion: HKc, Jacobi procedure")
        if save == 1
            savefig("HK_GS_5players.png")
            savefig("HK_GS_5players.eps")
        end
    end 
end
#############################################################
function plot_targ_10a(x_plot, T, model, save)
    figure(figsize=(6,6))
    grid()
    if model == 1
        for i in 1:10
            plot(x_plot[i,1:T],".-")
        end
    elseif model == 2
        for i in 11:20
            plot(x_plot[i,1:T],".-")
        end
    elseif model == 3
        for i in 21:30
            plot(x_plot[i,1:T],".-")
        end
    end
    xlabel("Time")
    ylabel("Opinions \$x_i\$")
    legend([L"x_1",L"x_2", L"x_3", L"x_4", L"x_5", L"x_6", L"x_7", L"x_8", L"x_9", L"x_{10}"],loc="right")
    
    if model == 1
        title("Time plots of opinion: dGc, Jacobi procedure")
        if save == 1
            savefig("DeGroot_Jacobi_10a.png")
            savefig("DeGroot_Jacobi_10a.eps")
        end
    elseif model == 2
        title("Time plots of opinion: FJc, Jacobi procedure")
        if save == 1
            savefig("FJ_Jacobi_10a.png")
            savefig("FJ_Jacobi_10a.eps")
        end
    elseif model == 3
        title("Time plots of opinion: HKc, Jacobi procedure")
        if save == 1
            savefig("HK_Jacobi_10a.png")
            savefig("HK_Jacobi_10a.eps")
        end
    end 
end

##########################################################
function equilibrium_(N_a,A,B,Θ,γ,targets,x_init,model)
    P = zeros(N_a,N_a)
    G = zeros(N_a,1)
    for i in 1:size(B,2)
        u = B[:,i]
        P = P + (1/(u'*u + γ[i]))*(u*u')
        G = G + ((1/(u'*u + γ[i]))*(u*u'))*targets[:,i]
    end
    if model == 1
        xeq = (I(N_a)-((I(N_a)-P)*A))\G
    elseif model == 2
        xeq = (I(N_a)-((I(N_a)-P)*A))\((I(N_a)-P)*Θ*x_init + G)
    end
    radius = maximum(abs.(eigvals((I(N_a)-P)*A)));
    Acl = (I(N_a)-P)*A; 
    return radius, xeq
end

equilibrium_ (generic function with 1 method)

# Mazalov comp

In [7]:
#Agents initial conditions
N_a_1 =  10 #number of agents
#x_init_1 = rand(N_a_1) 
#x_init_1 = 0.5*ones(N_a_1);
#x_init_1 = [0.5;0.7; 0.4; 0.4; 0.8; 0.7; 0.9; 0.6; 0.3; 0.5]
x_init_1 = [0.1;0.7; 0.4; 0.4; 0.8; 0.7; 0.9; 0.6; 0.9; 0.5]

#Players targets
N_p_1 = 4

x1_hat_1 = 0.5
x2_hat_1 = 0.7
x3_hat_1 = 0.2
x4_hat_1 = 0.3

p1 = [x1_hat_1;0;0;0;0;0;0;0;0;0]
p2 = [0;0;0;x2_hat_1;0;0;0;0;0;0]
p3 = [0;0;0;0;0;x3_hat_1;0;0;0;0]
p4 = [0;0;0;0;0;0;0;0;x4_hat_1;0]

targets_1 = [p1 p2 p3 p4]

#γ_1 = [0.01; 0.01; 0.01];
γ_1 = [0.01; 0.01; 0.01;0.01];
#println("Gamma = ", γ_1[1:N_p_1]);

#Dynamics
#DeGroot model
A_1 = 1/N_a_1.*ones(N_a_1,N_a_1);
B_1 = [1 0 0 0; 0 0 0 0; 0 0 0 0; 0 1 0 0; 0 0 0 0; 0 0 1 0;  0 0 0 0; 0 0 0 0; 0 0 0 1;  0 0 0 0];

#Friedkin-Johnsen model
Θ_1 = diagm([0.8,0.2,0.4,0,0.8,0,0.3,0.4,0.5,0])
A_FJ_1 = (I(N_a_1)-Θ_1)*A_1;

#Hegselmann-Krause model
F_1 = 90;
W_1 = 0.4

A_HK_1 = ones(N_a_1,N_a_1)

K_f_1 = 20; #horizon
T = K_f_1;

In [13]:
x_os_1dg, u_os_1dg = COD_OSA_J_DeGroot(x_init_1,targets_1,γ_1,K_f_1,A_1,B_1,N_a_1,N_p_1,0);
x_os_1fj, u_os_1fj = COD_OSA_J_targets_FJ(x_init_1,targets_1,γ_1,K_f_1,A_FJ_1,B_1,Θ_1,N_a_1,N_p_1,0);
x_os_1hk, u_os_1hk = COD_OSA_J_targets_HK(x_init_1,targets_1,γ_1,K_f_1,A_HK_1,B_1,F_1,W_1,0);
x_os = [x_os_1dg;x_os_1fj;x_os_1hk];

In [22]:
P = zeros(N_a_1,N_a_1)
G = zeros(N_a_1,1)
for i in 1:size(B_1,2)
    u = B_1[:,i]
    P = P + (1/(u'*u + γ_1[i]))*(u*u')
    G = G + ((1/(u'*u + γ_1[i]))*(u*u'))*targets_1[:,i]
end
xeq = (I(N_a_1)-((I(N_a_1)-P)*A_1))\G

10×1 Matrix{Float64}:
 0.49925742574257426
 0.4250000000000001
 0.42500000000000004
 0.6972772277227725
 0.4250000000000001
 0.2022277227722773
 0.42500000000000004
 0.4250000000000001
 0.3012376237623763
 0.42500000000000004

In [21]:
# Calculus for equilibrium point and spectral radius
model = 1; # 1 - DeGroot; 2 - FJ
s_radius, x_eq = equilibrium_(N_a_1,A_FJ_1,B_1,Θ_1,γ_1,targets_1, x_init_1, model);
x_eq

10×1 Matrix{Float64}:
 0.495598304532116
 0.22171503097489403
 0.1662862732311705
 0.695813304838798
 0.055428757743723495
 0.20076379988830317
 0.19400065210303225
 0.1662862732311705
 0.29840170192434956
 0.27714378871861756

In [ ]:
#x_plot = mov_avg(x_os_1fj, 1)

model = 3  # model: 1 - DeGroot; 2 - FJ; 3 - HK
save = 0
plot_targ_10a(x_os, T, model, save)

# 10 agents, with 2 palyers (polarization, dual market)

In [3]:
#Initial Conditions
x_init_2 = [1;0.7;1;0.9;1;-1;-0.8;-1;-0.8;-1]
#Players targets
N_p_2 = 2
x1_hat_2 = -1
x2_hat_2 = 1

targets_2 = [x1_hat_2  x2_hat_2;
             x1_hat_2     0;
             x1_hat_2     0;  
             x1_hat_2     0;  
             x1_hat_2  x2_hat_2;
               0       x2_hat_2;
               0       x2_hat_2;
               0       x2_hat_2;
               0       x2_hat_2;
             x1_hat_2  x2_hat_2]     
γ_2 = [0.01;0.01]
#println("Gamma = ", γ[1:N_p_s])

#Dynamics
B_2 = [1 1; 1 0; 1 0; 1 0; 1 1; 0 1; 0 1; 0 1; 0 1; 1 1]

A_t = [80 55 30 75 0 0 0 0 0 0;
       55  54 63 0 64 0 0 0 0 0 ;
       60 40 65 74 67 0 0 0 0 0 ;
       75 0 55 25 85 34 12 0 0 0 ;
       0 64 38 85 85 0 0 0 0 23 ;
       0 0 0 0 0 85 82 62 84 0 ;
       20 0 0 0 40 82 47 50 94 55 ;
       0 0 0 0 0 62 0 61 82 47 ;
       0 0 0 0 0 53 58 47 55 68 ;
       0 0 0 6 0 0 55 47 69 41]
N_a_2 = size(A_t,2)
A_2 = rs_ize(A_t)

#Friedkin-Johsen model
Θ_2 = diagm([0.5,0.3,0,0.7,0,0.1,0,0.3,0,0.6])
A_FJ_2 = (I(N_a_2)-Θ_2)*A_2;

#Hegselmann-Krause model
F_2 = 10;
W_2 = 1

A_HK_2 = [1 1 1 1 0 0 0 0 0 0;
          1 1 1 0 1 0 0 0 0 0;
          1 1 1 1 1 0 0 0 0 0;
          1 0 1 1 1 1 1 0 0 0;
          0 1 1 1 1 0 0 0 0 1;
          0 0 0 0 0 1 1 1 1 0;
          1 0 0 0 1 1 1 1 1 1;
          0 0 0 0 0 1 0 1 1 1;
          0 0 0 0 0 1 1 1 1 1;
          0 0 0 1 0 0 1 1 1 1]

K_f_2 = 200; #horizon
T_2 = K_f_2;

In [4]:
x_os_2dg, u_os_2dg = COD_OSA_J_DeGroot(x_init_2,targets_2,γ_2,K_f_2,A_2,B_2,N_a_2,N_p_2,0);
x_os_2fj, u_os_2fj = COD_OSA_J_targets_FJ(x_init_2,targets_2,γ_2,K_f_2,A_FJ_2,B_2,Θ_2,N_a_2,N_p_2,0);
x_os_2hk, u_os_2hk = COD_OSA_J_targets_HK(x_init_2,targets_2,γ_2,K_f_2,A_HK_2,B_2,F_2,W_2,0);
x_os_2 = [x_os_2dg;x_os_2fj;x_os_2hk];


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



In [5]:
# Calculus for equilibrium point and spectral radius
model = 1; # 1 - DeGroot; 2 - FJ
s_radius_2, x_eq_2 = equilibrium_point(N_a_2,A_FJ_2,B_2,Θ_2,γ_2,targets_2, x_init_2, model);
x_eq_2

10×1 Matrix{Float64}:
 -0.5661185472669902
 -1.3196370577200864
 -1.5354095977414082
 -0.6838258177040537
 -1.0775289409162645
  1.920702581593011
  1.5296398197647485
  1.437596197018585
  1.770379111389804
  0.40789436170408494

In [6]:
x_os_2dg[:,end]

10-element Vector{Float64}:
 -1.5443659014414213
 -1.7406198968612037
 -1.6253925221613341
 -1.0936032432766953
 -1.2102712283736248
  1.9924189917986959
  1.4946684620657487
  2.052616466241989
  1.9324897325353394
  1.6201367415852745

In [ ]:
model = 1  # model: 1 - DeGroot; 2 - FJ; 3 - HK
save = 0
plot_targ_10a(x_os_2, T_2, model, save)

# CODES USE

## 5 agents, 3 players
#Agents initial conditions
N_a_1 =  5 #number of agents
#x_init_1 = rand(N_a_1) 
x_init_1 = 0.5*ones(N_a_1);
#println("The initial conditions are ", x_init_1)

#Players targets
N_p_1 = 3
x1_hat_1 = 0.2
x2_hat_1 = 0.6
x3_hat_1 = 0.9

targets_1 = [x1_hat_1    0        0          ; #first agent
             0      x2_hat_1    0          ; 
             0       0       x3_hat_1      ;
             0       0        0          ;
            0        0        0          ] #last agent

#γ_1 = 0.1*rand(size(targets_1,1))
γ_1 = [0.01; 0.01; 0.01];
#println("Gamma = ", γ_1[1:N_p_1]);

#Dynamics
#DeGroot model
#A_1 = [0.1 0.4 0.2 0.1 0.2; 0.5 0.1 0.1 0.1 0.2; 0.6 0.1 0.1 0.1 0.1; 0.1 0.1 0.3 0.4 0.1; 0.2 0.2 0.1 0.1 0.4];
A_1 = [0.1 0.4 0.2 0.1 0.2; 0.5 0.1 0.1 0.1 0.2; 0.3 0.1 0.4 0.1 0.1; 0.1 0.1 0.3 0.4 0.1; 0.2 0.05 0.1 0.45 0.2];
B_1 = [0.8 0 0; 0 0.8 0; 0 0 0.8; 0 0 0; 0 0 0];

#Friedkin-Johnsen model
Θ_1 = diagm([0.8,0.2,0.4,0,0.8])
A_FJ_1 = (I(N_a_1)-Θ_1)*A_1;

#Hegselmann-Krause model
F_1 = 10;
W_1 = 0.2

A_HK_1 = ones(N_a_1,N_a_1)

K_f_1 = 60; #horizon

## 10 agents, 2 players
#Initial Conditions
x_init_2 = [1;0.7;1;0.9;1;-1;-0.8;-1;-0.8;-1]
#Players targets
N_p_2 = 2
x1_hat_2 = -1
x2_hat_2 = 1

targets_2 = [x1_hat_2  x2_hat_2;
             x1_hat_2     0;
             x1_hat_2     0;  
             x1_hat_2     0;  
             x1_hat_2  x2_hat_2;
               0       x2_hat_2;
               0       x2_hat_2;
               0       x2_hat_2;
               0       x2_hat_2;
             x1_hat_2  x2_hat_2]     
γ_2 = [0.01;0.01]
#println("Gamma = ", γ[1:N_p_s])

#Dynamics
B_2 = [0.5 0.5; 0.5 0; 0.5 0; 0.5 0; 0.5 0.5; 0 0.5; 0 0.5; 0 0.5; 0 0.5; 0.5 0.5]

A_t = [80 55 30 75 0 0 0 0 0 0;
       55  54 63 0 64 0 0 0 0 0 ;
       60 40 65 74 67 0 0 0 0 0 ;
       75 0 55 25 85 34 12 0 0 0 ;
       0 64 38 85 85 0 0 0 0 23 ;
       0 0 0 0 0 85 82 62 84 0 ;
       20 0 0 0 40 82 47 50 94 55 ;
       0 0 0 0 0 62 0 61 82 47 ;
       0 0 0 0 0 53 58 47 55 68 ;
       0 0 0 6 0 0 55 47 69 41]
N_a_2 = size(A_t,2)
A_2 = rs_ize(A_t)

#Friedkin-Johsen model
Θ_2 = diagm([0.5,0.3,0,0.7,0,0.1,0,0.3,0,0.6])
A_FJ_2 = (I(N_a_2)-Θ_2)*A_2;

#Hegselmann-Krause model
F_2 = 10;
W_2 = 1

A_HK_2 = [1 1 1 1 0 0 0 0 0 0;
          1 1 1 0 1 0 0 0 0 0;
          1 1 1 1 1 0 0 0 0 0;
          1 0 1 1 1 1 1 0 0 0;
          0 1 1 1 1 0 0 0 0 1;
          0 0 0 0 0 1 1 1 1 0;
          1 0 0 0 1 1 1 1 1 1;
          0 0 0 0 0 1 0 1 1 1;
          0 0 0 0 0 1 1 1 1 1;
          0 0 0 1 0 0 1 1 1 1]

K_f_2 = 60; #horizon